In [1]:
from dotenv import load_dotenv
load_dotenv()
from rtai.persona.agent import Agent
from rtai.persona.agent_manager import AgentManager
from queue import Queue
from rtai.utils.config import YamlLoader
from rtai.llm.llm_client import LLMClient
from typing import List, Tuple

In [2]:
class TestAgent(Agent):
    def __init__(self, agent_mgr: AgentManager, event_queue: Queue, client: LLMClient, file_path: str=""):
        super().__init__(agent_mgr, event_queue, client, file_path)
    
    def _create_day_plan(self, new_day: bool, first_day: bool) -> None:
        wake_up_hour = self.s_mem.generate_wake_up_hour()
        daily_req = self.generate_daily_plan() # TODO
        return self.generate_hourly_schedule(wake_up_hour, daily_req) # TODO

    def generate_daily_plan(self) -> str:
        """   Generates the daily plan for the persona. 
        Basically the long term planning that spans a day. Returns a list of actions
        that the persona will take today. Usually comes in the following form: 
        'wake up and complete the morning routine at 6:00 am', 
        'eat breakfast at 7:00 am',.. 
        Note that the actions come without a period. 

        Persona state: identity stable set, lifestyle, cur_data_str, first_name

        INPUT: 
            persona: The Persona class instance 
            wake_up_hour: an integer that indicates when the hour the persona wakes up 
                        (e.g., 8)
        OUTPUT: 
            a list of daily actions in broad strokes.
        EXAMPLE OUTPUT: 
            ['wake up and complete the morning routine at 6:00 am', 
            'have breakfast and brush teeth at 6:30 am',
            'work on painting project from 8:00 am to 12:00 pm', 
            'have lunch at 12:00 pm', 
            'take a break and watch TV from 2:00 pm to 4:00 pm', 
            'work on painting project from 4:00 pm to 6:00 pm', 
            'have dinner at 6:00 pm', 'watch TV from 7:00 pm to 8:00 pm']"""
        
        PROMPT = """Based on the description of a person, create a daily plan for that person with a list of actions that the persona will take today. Output the plans in a list separated by commas.
        """

        return self.llm_client.generate_from_prompt(system_prompt=PROMPT, user_prompt=self.get_common_set_str())


    def generate_hourly_schedule(self, wake_up_hour, daily_req) -> List[Tuple]:
        """ * Uses LLM
        TODO
        Implement function to generate an hourly schedule for the day using LLMs
        Try and moving to a memory class

        OUTPUT:
        (activity, start_time, duration)


        """
        hour_str = ["00:00 AM", "01:00 AM", "02:00 AM", "03:00 AM", "04:00 AM", 
                    "05:00 AM", "06:00 AM", "07:00 AM", "08:00 AM", "09:00 AM", 
                    "10:00 AM", "11:00 AM", "12:00 PM", "01:00 PM", "02:00 PM", 
                    "03:00 PM", "04:00 PM", "05:00 PM", "06:00 PM", "07:00 PM",
                    "08:00 PM", "09:00 PM", "10:00 PM", "11:00 PM"]
        
        PROMPT = f"""
                Based on the description of a person and their high level plan for the day, create an hourly schedule for that person with a list of actions.
                Output the plans in a list separated by commas. Every plan should have an activity, start time, and duration!
                Here are the hours that you should use for scheduling: {hour_str}
                """
        return self.llm_client.generate_from_prompt(system_prompt=PROMPT, user_prompt=self.get_common_set_str() + "\n\n Daily Plan:" + daily_req)


In [3]:
cfg = YamlLoader.load("configs/rtai.yaml")
event_queue = Queue()
llm_client = LLMClient(cfg.expand('LLMClient'))
mgr = AgentManager(event_queue, cfg.expand('Agents'), client=llm_client)
a = TestAgent(mgr, event_queue, llm_client, file_path='tests/samples/personas/persona2.txt')

In [4]:
wake_up_hour = a.s_mem.generate_wake_up_hour()
daily_req = a.generate_daily_plan() # TODO
sched = a.generate_hourly_schedule(wake_up_hour, daily_req) # TODO

KeyboardInterrupt: 

In [11]:
!pip3 install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 37.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.26-cp312-cp312-macosx_14_0_arm64.whl size=1812885 sha256=f121be113803910b5f0ccdbcac802e8b1e5a95e4df21cfd20e1b89dccd5b4ff8
  Stored in directory: /Users/nyeung/Library/Caches/pip/wheels/13/27/79/ea6fad0626516a5bf03d20de47f9df4d8da4b5dd6063d8e6c2
Successfully built llama-cpp-python


In [22]:
from guidance import models, instruction, gen
import guidance
MODEL_PATH = "/Users/nyeung/Projects/llama.cpp/models/mistral-7b-instruct-v0.1.Q5_K_S.gguf"

# load the model
llama2 = models.LlamaCpp(MODEL_PATH, n_gpu_layers=-1)


In [27]:
query = "Who won the last Kentucky derby and by how much? Give the answer in metric"
lm = llama2 + f'''\
Q: {query}
A: {gen(name="answer", stop="")}'''

In [26]:
lm["answer"]

'The last Kentucky Derby was held on May 7, 2021. The winner was Medina Spirit, trained by Eric Guillary and ridden by jockey John Velazquez. Medina Spirit won by 1/6th of a length over Mandaloun, who finished second.'

In [18]:
PROMPT_PLAN = """### Instruction:

Example for plan:
Here is {{name}}'s plan from now at 7:14:
[From 7:14 to 7:45]: Wake up and complete the morining routine
[From 7:45 to 8:35]: Eat breakfirst
[From 8:35 to 17:10]: Go to school and study
[From 17:10 to 22:30]: Play CSGO
[From 22:30 to 7:30]: Go to sleep

### Input:
Today is {{current_time}}. Please make a plan today for {{name}} in broad strokes. Given the summary:
{{summary}}

### Response:
Here is {{name}}'s plan from now at {{current_time}}:
[From {{now}} to {{gen 'to' pattern='[0-9]+:[0-9][0-9]' stop=']'}}]: {{gen 'task' top_k=30 top_p=0.18 repetition_penalty=1.15 temperature=1.99 stop='\n'}}
{{#geneach 'items' num_iterations=3}}[From {{gen 'this.from' pattern='[0-9]+:[0-9][0-9]' stop=' '}} to {{gen 'this.to' pattern='[0-9]+:[0-9][0-9]' stop=']'}}]: {{gen 'this.task' top_k=30 top_p=0.18 repetition_penalty=1.15 temperature=1.99 stop='\n'}}
{{/geneach}}"""
guidance(PROMPT_PLAN)

TypeError: module, class, method, function, traceback, frame, or code object was expected, got str

In [15]:
PROMPT_PLAN = """### Instruction:

Example for plan:
Here is {{name}}'s plan from now at 7:14:
[From 7:14 to 7:45]: Wake up and complete the morining routine
[From 7:45 to 8:35]: Eat breakfirst
[From 8:35 to 17:10]: Go to school and study
[From 17:10 to 22:30]: Play CSGO
[From 22:30 to 7:30]: Go to sleep

### Input:
Today is {{current_time}}. Please make a plan today for {{name}} in broad strokes. Given the summary:
{{summary}}

### Response:
Here is {{name}}'s plan from now at {{current_time}}:
[From {{now}} to {{gen 'to' pattern='[0-9]+:[0-9][0-9]' stop=']'}}]: {{gen 'task' top_k=30 top_p=0.18 repetition_penalty=1.15 temperature=1.99 stop='\n'}}
{{#geneach 'items' num_iterations=3}}[From {{gen 'this.from' pattern='[0-9]+:[0-9][0-9]' stop=' '}} to {{gen 'this.to' pattern='[0-9]+:[0-9][0-9]' stop=']'}}]: {{gen 'this.task' top_k=30 top_p=0.18 repetition_penalty=1.15 temperature=1.99 stop='\n'}}
{{/geneach}}"""
lm = llama2(PROMPT_PLAN)
guidance.llm = lm
result = lm({'name': 'Nathan', 'summary': 'Nathan is a student who is going to school today. He will study and play CSGO.'})

TypeError: 'generator' object is not callable

In [12]:
print(sched.split(', '))

['"Wake up at 7:00 AM', 'have breakfast with Lois Lane', 'exercise for 30 minutes', 'attend press conference', 'write article', 'meet with Lois to review', 'visit Justice League', 'lunch with family', 'reflect on heritage', 'conduct interviews', 'attend charity event', 'dinner with Lois', 'rest."']
